In [1]:
import torch
import json
import torch.nn as nn
import numpy as np
from math import floor
from tqdm import tqdm

from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import ViTImageProcessor, ViTForImageClassification, TrainingArguments, Trainer, DefaultDataCollator
from datasets import load_dataset, Dataset

2024-12-12 23:24:56.660232: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-12 23:24:56.703114: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-12 23:24:56.703152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-12 23:24:56.704405: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-12 23:24:56.712557: I tensorflow/core/platform/cpu_feature_guar

In [2]:
device = "cuda" if torch.cuda.is_available else "cpu"
class GlowViT(ViTForImageClassification):
    def help():
        print(ViTForImageClassification.__doc__)

In [3]:
yalu_ds_list = [("SeaSponge/wildme10_classify", "glow-vit"),
                ("yin30lei/wildlife_very_dark", "glow-vit-dark"),
                ("yin30lei/wildlife_well_illuminated", "glow-vit-illuminate"),
                ("yin30lei/wildlife_mixed", "glow-vit-mix")]

# yalu_ds = yalu_ds_list[1][0]
yalu_model_name = yalu_ds_list[3][1]
# wildlife_test_ds = load_dataset(yalu_ds, cache_dir=Path.cwd() / "yalu_dataset", split="test" , num_proc=2)

In [4]:
# 11 labels for wildme10_classify
num_labels = 11
# print(wildlife_test_ds[0])

In [5]:
label2id = {"lion": 0, "raccoon": 1, "tiger": 2, "wolf": 3,
            "bear": 4, "hare": 5, "fox": 6, "deer": 7, 
            "leopard" : 8, "hyena": 9, "antelope": 10}


id2label = {v:k for k, v in label2id.items()}

In [6]:
yalu_checkpoint = f"SeaSponge/{yalu_model_name}"
model = GlowViT.from_pretrained(yalu_checkpoint,
                cache_dir=yalu_model_name,
                label2id=label2id,
                id2label=id2label,
                num_labels=num_labels,
                attn_implementation="sdpa") # no flash attention yet for ViT model

image_processor = ViTImageProcessor.from_pretrained(yalu_checkpoint)

In [7]:
def transform_ds(examples):
    images, labels = [], []
    for image, label in zip(examples["image"], examples["labels"]):
        pix_val = image_processor(images=image.convert("RGB"), return_tensors="pt")["pixel_values"].squeeze(0)
        pix_val.to(device)
        #! supposed to be a number here
        label = label2id[label]
        images.append(pix_val)
        labels.append(label)

    return {"pixel_values": images, "labels": labels}

In [8]:
# Define the testing arguments

testing_args = TrainingArguments(
    output_dir=yalu_model_name,
    per_device_eval_batch_size=16,
    logging_steps=30,
    save_total_limit=1,
    remove_unused_columns=False,
    push_to_hub=True,
)

# Define the trainer

trainer = Trainer(
    model=model,
    args=testing_args,
    # data_collator=data_collator,
    # eval_dataset=test_ds,
    tokenizer=image_processor,
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [9]:
# main_ds = ["SeaSponge/wildme10_classify", "yin30lei/wildlife_very_dark",
#                  "yin30lei/wildlife_well_illuminated", "yin30lei/wildlife_mixed"]

# for ds in main_ds:
#     main_test_ds = load_dataset(ds, cache_dir=Path.cwd() / "yalu_dataset", split="test" , num_proc=2)
#     test_ds = main_test_ds.with_transform(transform_ds)
    
#     metrics = trainer.evaluate(test_ds)
#     trainer.log_metrics(f"{yalu_model_name} | testing on {ds} ", metrics)

You can use `tensorboard --logdir <logging directory>` to get more information.

In [10]:
# additional_ds = ["yin30lei/wildlife_very_dark_test", "yin30lei/wildlife_grayscale",
#                  "yin30lei/wildlife_less_saturated", "yin30lei/wildlife_underexposed"]
# for ds in additional_ds:
#     additional_test_ds = load_dataset(ds, cache_dir=Path.cwd() / "yalu_dataset", split="train" , num_proc=2)
#     test_ds = additional_test_ds.with_transform(transform_ds)
    
#     metrics = trainer.evaluate(test_ds)
#     trainer.log_metrics(f"{yalu_model_name} | testing on {ds} ", metrics)

In [11]:
# verydark_filter = ["yin30lei/wildlife_verydark_clahe", "yin30lei/wildlife_verydark_he",
#                  "yin30lei/wildlife_verydark_afifi"]
# for ds in verydark_filter:
#     additional_test_ds = load_dataset(ds, cache_dir=Path.cwd() / "yalu_dataset", split="train" , num_proc=2)
#     test_ds = additional_test_ds.with_transform(transform_ds)
    
#     metrics = trainer.evaluate(test_ds)
#     trainer.log_metrics(f"{yalu_model_name} | testing on {ds} ", metrics)

In [12]:
# underexposed_filter = ["yin30lei/wildlife_underexposed_clahe", "yin30lei/wildlife_underexposed_he",
#                  "yin30lei/wildlife_underexposed_afifi"]
# for ds in underexposed_filter:
#     additional_test_ds = load_dataset(ds, cache_dir=Path.cwd() / "yalu_dataset", split="train" , num_proc=2)
#     test_ds = additional_test_ds.with_transform(transform_ds)
    
#     metrics = trainer.evaluate(test_ds)
#     trainer.log_metrics(f"{yalu_model_name} | testing on {ds} ", metrics)

In [13]:
less_saturated_filter = ["yin30lei/wildlife_less_saturated_clahe", "yin30lei/wildlife_less_saturated_he",
                 "yin30lei/wildlife_less_saturated_afifi"]
for ds in less_saturated_filter:
    additional_test_ds = load_dataset(ds, cache_dir=Path.cwd() / "yalu_dataset", split="train" , num_proc=2)
    test_ds = additional_test_ds.with_transform(transform_ds)
    
    metrics = trainer.evaluate(test_ds)
    trainer.log_metrics(f"{yalu_model_name} | testing on {ds} ", metrics)

***** glow-vit-mix | testing on yin30lei/wildlife_less_saturated_clahe  metrics *****
  eval_loss                   =      0.207
  eval_model_preparation_time =     0.0029
  eval_runtime                = 0:00:59.91
  eval_samples_per_second     =     18.978
  eval_steps_per_second       =      1.202
***** glow-vit-mix | testing on yin30lei/wildlife_less_saturated_he  metrics *****
  eval_loss                   =     0.1004
  eval_model_preparation_time =     0.0029
  eval_runtime                = 0:00:52.45
  eval_samples_per_second     =     21.674
  eval_steps_per_second       =      1.373
***** glow-vit-mix | testing on yin30lei/wildlife_less_saturated_afifi  metrics *****
  eval_loss                   =     0.0977
  eval_model_preparation_time =     0.0029
  eval_runtime                = 0:00:52.51
  eval_samples_per_second     =     21.653
  eval_steps_per_second       =      1.371
